### Instaladores

In [82]:
# ------------------------- #
# Importación de librerías  #
# ------------------------- #

# Librerías estándar para análisis de datos y cálculos matemáticos
import numpy as np
import pandas as pd
import math
import random

# Librerías para bases de datos
import pyodbc
import sqlite3

# Visualización
import seaborn as sns

# Machine Learning y modelado
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from imblearn.over_sampling import RandomOverSampler
from sklearn.cluster import KMeans
from sklearn.neighbors import LocalOutlierFactor

# Estadísticas y pruebas de hipótesis
from scipy.stats import (
    rv_discrete, t, binom, chi2, f, 
    ttest_ind
)
from sklearn.utils import resample

# ------------------------- #
# Importación de módulos    #
# ------------------------- #

# Modulo EDA
import importlib
import funciones_eda
importlib.reload(funciones_eda)
from funciones_eda import *

# Módulo de funciones personalizadas
import funciones_m
importlib.reload(funciones_m)
from funciones_m import *

# Módulo de distribución
import funciones_distribucion
importlib.reload(funciones_distribucion)
from funciones_distribucion import *

# Módulo de pruebas significativas
import funciones_pruebas_significativas
importlib.reload(funciones_pruebas_significativas)
from funciones_pruebas_significativas import *

# Módulo de clostering
import funciones_clostering
importlib.reload(funciones_clostering)
from funciones_clostering import *


### data set 

In [83]:
url = 'df_modelo.csv'
df = pd.read_csv(url, index_col=None)
#df.drop('index', axis=1, inplace=True)

In [84]:
df=df[['Semana','CUSTOMER_ID','cliente','PRODUCT_ID','producto','QTY']]

### target encoding 

Target encoding es una técnica de codificación utilizada en machine learning para transformar variables categóricas en valores numéricos mediante el uso de la variable objetivo. 
Se basa en calcular el promedio o estadístico de la variable objetivo para cada categoría de la variable categórica y reemplazar dicha categoría por su valor numérico correspondiente

cliente

In [85]:
customer_target_mean = df.groupby('CUSTOMER_ID')['QTY'].mean().reset_index()
customer_target_mean.columns = ['CUSTOMER_ID', 'customer_target_mean']
df = df.merge(customer_target_mean, on='CUSTOMER_ID', how='left')

producto

In [86]:
product_target_mean = df.groupby('PRODUCT_ID')['QTY'].mean().reset_index()
product_target_mean.columns = ['PRODUCT_ID', 'product_target_mean']
df = df.merge(product_target_mean, on='PRODUCT_ID', how='left')

### crear variable  target categorica se_compra

In [87]:
df_model=df.copy()

se organiza dataframe en el orden cliente, producto y semana

In [88]:
df_model=df_model.sort_values(by=['CUSTOMER_ID','PRODUCT_ID','Semana'])

n términos generales, esta línea de código crea una nueva columna que contiene el valor de la siguiente fila dentro de cada grupo definido por las columnas CUSTOMER_ID y PRODUCT_ID.

El objetivo es "desplazar" los valores de una columna ( QTYen este caso) hacia arriba dentro de un grupo, lo que permite relacionar el valor actual con el siguiente valor temporal.

In [89]:
df_model['QTY_next_week'] = df_model.groupby(['CUSTOMER_ID', 'PRODUCT_ID'])['QTY'].shift(-1)

 Este código crea una nueva columna en el DataFrame llamada QTY_next_week, que contiene el valor de QTY de la semana siguiente para el mismo cliente (CUSTOMER_ID) y el mismo producto (PRODUCT_ID). 

In [90]:
df_model['Semana_next'] = df_model.groupby(['CUSTOMER_ID', 'PRODUCT_ID'])['Semana'].shift(-1)

**se crea la variable de compra proxima semana**

In [91]:
df_model['se_compra'] = ((df_model['Semana_next'] == df_model['Semana'] + 1) & 
                         (df_model['QTY_next_week'] > 0)).astype(int)

se filtra data set

In [92]:
df_model_=df_model[['customer_target_mean','product_target_mean','Semana','QTY','se_compra']]

In [93]:
df_model_ = df_model_.rename(columns={
    'customer_target_mean': 'target_cliente',
    'product_target_mean': 'target_producto',
    'QTY': 'cantidad',
})


### datos de prueba

se está seleccionando el 90 % del DataFrame y dejando las restantes, por lo que la división no es aleatoria sino secuencial y dejando el 10% para prueba del modelo.

In [94]:
percentage_first_part = 0.90

# sacamos el porcentaje de datos de la primera parte
n_rows_part_1= int(len(df_model_)*percentage_first_part)

# sacamos el 90% de los datos aleatorios 
df_ = df_model_.iloc[:n_rows_part_1].reset_index(drop=True)
df_p=df_model_.iloc[n_rows_part_1:].reset_index(drop=True)


In [95]:
len(df_)

18699

Eliminar outliers

**Valores atípicos**

In [96]:
# crear un objeto LocalOutlierFactor con n_neighbors=13
lof = LocalOutlierFactor(n_neighbors=13, contamination=0.1)
y_pred = lof.fit_predict(df_[['cantidad']])

# Identificar los índices de los valores atípicos
outliers = y_pred == -1

In [97]:
print('Indices de valores atípicos: ', df_[outliers].index.tolist())

Indices de valores atípicos:  [350, 1563, 2777, 4628, 5363, 5671, 6169, 6459, 6622, 8408, 8548, 8936, 9078, 9290, 9291, 9855, 10481, 10777, 11849, 12020, 12050, 12070, 12315, 13158, 13714, 13719, 13761, 13842, 14661, 16266, 17793, 17797]


In [98]:
# Eliminar los valores atípicos del dataframe original
df_=df_.loc[~outliers]

### oversampler

In [99]:
# Se crea un objeto RandomOverSampler que permite crear copias aleatorias de las muestras de la clase 
# minoritaria para igualar el número de muestras entre las diferentes clases.
oversampler = RandomOverSampler()

# Se realiza el sobremuestreo para las variables de entrada y salida
X_resampled, y_resampled = oversampler.fit_resample(df_.drop('se_compra', axis=1), df_['se_compra'])

# Se crean dos dataframes con las variables de entrada y otro con la variable de salida a partir del proceso anterior
df_1 = pd.DataFrame(X_resampled, columns=df_.columns.drop('se_compra'))
df_2 = pd.DataFrame(y_resampled)

# Se concatenan los dataframes horizontalmente
df__balanced = pd.concat([df_1, df_2], axis=1)

df_=df__balanced

In [100]:
df_.to_csv('df_ML.csv', index=False)